In [ ]:
import numpy as np
import torch
import torch.optim as optim
import pandas as pd


#서울시 우리마을가게 상권분석서비스(자치구별 상권변화지표)

#encoding= 'CP949' : 한글을 읽어들이기 위해서.

df=pd.read_csv('서울시 우리마을가게 상권분석서비스(자치구별 상권변화지표).csv', encoding= 'CP949')
df


,기준_년_코드,기준_분기_코드,시군구_코드,시군구_코드_명,상권_변화_지표,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2019,4,11740,강동구,LL,다이나믹,105,50,110,54
1,2019,4,11710,송파구,LL,다이나믹,110,53,110,54
2,2019,4,11680,강남구,LL,다이나믹,106,50,110,54
3,2019,4,11650,서초구,HH,정체,118,54,110,54
4,2019,4,11620,관악구,LL,다이나믹,108,52,110,54
...,...,...,...,...,...,...,...,...,...,...
608,2014,1,11215,광진구,LL,다이나믹,89,44,91,44
609,2014,1,11200,성동구,HH,정체,93,46,91,44
610,2014,1,11170,용산구,HH,정체,101,47,91,44
611,2014,1,11140,중구,HH,정체,114,49,91,44


In [ ]:
df['시군구_코드_명'].unique()   # 25개의 자치구

array(['강동구', '송파구', '강남구', '서초구', '관악구', '동작구', '영등포구', '금천구', '구로구',
       '강서구', '양천구', '마포구', '서대문구', '은평구', '노원구', '도봉구', '강북구', '성북구',
       '중랑구', '동대문구', '광진구', '성동구', '용산구', '중구', '종로구'], dtype=object)

## 1. 데이터 전처리
- 불필요한 컬럼, 예측에 방해되는 컬럼 제거
- 텍스트 숫자로 변경
- 의미있는 데이터 만들기
- 컬럼명 변경
- 결측치확인 및 제거

In [ ]:
df.columns

Index(['기준_년_코드', '기준_분기_코드', '시군구_코드', '시군구_코드_명', '상권_변화_지표', '상권_변화_지표_명',
       '운영_영업_개월_평균', '폐업_영업_개월_평균', '서울_운영_영업_개월_평균', '서울_폐업_영업_개월_평균'],
      dtype='object')

In [ ]:
# 시군구 코드와 시군구 코드명은 같은 정보를 가지고 있으므로 둘 중 하나를 제거
del df['시군구_코드_명']

# 상권변화지표와 상권변화지표명은 같은 정보를 가지고 있으므로 둘 중 하나를 제거
del df['상권_변화_지표']

In [ ]:
df.head()

,기준_년_코드,기준_분기_코드,시군구_코드,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2019,4,11740,다이나믹,105,50,110,54
1,2019,4,11710,다이나믹,110,53,110,54
2,2019,4,11680,다이나믹,106,50,110,54
3,2019,4,11650,정체,118,54,110,54
4,2019,4,11620,다이나믹,108,52,110,54


* ['상권 변화지표명'] 컬럼 데이터를 숫자로 변경
  - 상권 변화: 다이나믹(3) 상권확장(2) 상권축소(1) 정체(0)

In [ ]:
df['상권_변화_지표_명']=df['상권_변화_지표_명'].replace(['다이나믹','상권확장','상권축소','정체'],[3,2,1,0])
df.head()

,기준_년_코드,기준_분기_코드,시군구_코드,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균
0,2019,4,11740,3,105,50,110,54
1,2019,4,11710,3,110,53,110,54
2,2019,4,11680,3,106,50,110,54
3,2019,4,11650,0,118,54,110,54
4,2019,4,11620,3,108,52,110,54


- ['운영_영업_개월_평균']와	['폐업_영업_개월_평균'] 를 이용해 의미있는 데이터 컬럼 만들기

  - 서울 대비 해당 도시의 영업개월 평균
  - 서울 대비 해당 도시의 폐업개월 평균


In [ ]:
df['서울 대비 운영 영업개월 평균']=df['운영_영업_개월_평균']/df['서울_운영_영업_개월_평균']
df['서울 대비 운영 영업개월 평균'].head()

0    0.954545
1    1.000000
2    0.963636
3    1.072727
4    0.981818
Name: 서울 대비 운영 영업개월 평균, dtype: float64

In [ ]:
df['서울 대비 폐업 영업개월 평균']=df['폐업_영업_개월_평균']/df['서울_폐업_영업_개월_평균']
df['서울 대비 폐업 영업개월 평균'].head()

0    0.925926
1    0.981481
2    0.925926
3    1.000000
4    0.962963
Name: 서울 대비 폐업 영업개월 평균, dtype: float64

In [ ]:
df.head()

,기준_년_코드,기준_분기_코드,시군구_코드,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울_운영_영업_개월_평균,서울_폐업_영업_개월_평균,서울 대비 운영 영업개월 평균,서울 대비 폐업 영업개월 평균
0,2019,4,11740,3,105,50,110,54,0.954545,0.925926
1,2019,4,11710,3,110,53,110,54,1.000000,0.981481
2,2019,4,11680,3,106,50,110,54,0.963636,0.925926
3,2019,4,11650,0,118,54,110,54,1.072727,1.000000
4,2019,4,11620,3,108,52,110,54,0.981818,0.962963


In [ ]:
del df['서울_운영_영업_개월_평균']
del df['서울_폐업_영업_개월_평균']
df.head()

,기준_년_코드,기준_분기_코드,시군구_코드,상권_변화_지표_명,운영_영업_개월_평균,폐업_영업_개월_평균,서울 대비 운영 영업개월 평균,서울 대비 폐업 영업개월 평균
0,2019,4,11740,3,105,50,0.954545,0.925926
1,2019,4,11710,3,110,53,1.000000,0.981481
2,2019,4,11680,3,106,50,0.963636,0.925926
3,2019,4,11650,0,118,54,1.072727,1.000000
4,2019,4,11620,3,108,52,0.981818,0.962963


In [ ]:
# 컬럼명 변경

df.columns=['year','quarter','city','Commercial change','Operating months','Closing months', 'Operating months_Average', 'Closing months_Average']
df.head()

,year,quarter,city,Commercial change,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2019,4,11740,3,105,50,0.954545,0.925926
1,2019,4,11710,3,110,53,1.000000,0.981481
2,2019,4,11680,3,106,50,0.963636,0.925926
3,2019,4,11650,0,118,54,1.072727,1.000000
4,2019,4,11620,3,108,52,0.981818,0.962963


In [ ]:
# 결측치는 없다.

df.isna().sum()

year                        0
quarter                     0
city                        0
Commercial change           0
Operating months            0
Closing months              0
Operating months_Average    0
Closing months_Average      0
dtype: int64

## 2. data split & 파일만들기

- train.csv
- test.csv
- solution.csv
- submission.csv

- x와 labeled data(y)로 나누기

In [ ]:
xy_data=df.copy()

y_data=xy_data['Commercial change']

del xy_data['Commercial change']
x_data=xy_data

In [ ]:
x_data.head()

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2019,4,11740,105,50,0.954545,0.925926
1,2019,4,11710,110,53,1.000000,0.981481
2,2019,4,11680,106,50,0.963636,0.925926
3,2019,4,11650,118,54,1.072727,1.000000
4,2019,4,11620,108,52,0.981818,0.962963


In [ ]:
y_data.head()

0    3
1    3
2    3
3    0
4    3
Name: Commercial change, dtype: int64

- train과 test셋 나누기

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

x_data=np.array(x_data)
y_data=np.array(y_data)

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data,y_data,test_size=0.1, random_state=0)

In [ ]:
x_train

array([[2.01600000e+03, 1.00000000e+00, 1.12900000e+04, ...,
        5.10000000e+01, 1.03125000e+00, 1.06250000e+00],
       [2.01900000e+03, 4.00000000e+00, 1.12900000e+04, ...,
        5.60000000e+01, 1.00909091e+00, 1.03703704e+00],
       [2.01500000e+03, 4.00000000e+00, 1.13800000e+04, ...,
        4.80000000e+01, 9.68421053e-01, 1.00000000e+00],
       ...,
       [2.01600000e+03, 3.00000000e+00, 1.12000000e+04, ...,
        5.10000000e+01, 9.89583333e-01, 1.04081633e+00],
       [2.01800000e+03, 1.00000000e+00, 1.16200000e+04, ...,
        5.00000000e+01, 9.90000000e-01, 1.00000000e+00],
       [2.01400000e+03, 3.00000000e+00, 1.12000000e+04, ...,
        4.70000000e+01, 1.01075269e+00, 1.04444444e+00]])

In [ ]:
len(y_train)

551

In [ ]:
y_train=y_train.reshape(-1,1)

train=np.hstack([x_train, y_train])
train

array([[2.01600000e+03, 1.00000000e+00, 1.12900000e+04, ...,
        1.03125000e+00, 1.06250000e+00, 0.00000000e+00],
       [2.01900000e+03, 4.00000000e+00, 1.12900000e+04, ...,
        1.00909091e+00, 1.03703704e+00, 0.00000000e+00],
       [2.01500000e+03, 4.00000000e+00, 1.13800000e+04, ...,
        9.68421053e-01, 1.00000000e+00, 2.00000000e+00],
       ...,
       [2.01600000e+03, 3.00000000e+00, 1.12000000e+04, ...,
        9.89583333e-01, 1.04081633e+00, 2.00000000e+00],
       [2.01800000e+03, 1.00000000e+00, 1.16200000e+04, ...,
        9.90000000e-01, 1.00000000e+00, 3.00000000e+00],
       [2.01400000e+03, 3.00000000e+00, 1.12000000e+04, ...,
        1.01075269e+00, 1.04444444e+00, 0.00000000e+00]])

In [ ]:
train=pd.DataFrame(train,columns=['year','quarter','city','Operating months','Closing months', 'Operating months_Average', 'Closing months_Average','Commercial change'])
train

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average,Commercial change
0,2016.0,1.0,11290.0,99.0,51.0,1.031250,1.062500,0.0
1,2019.0,4.0,11290.0,111.0,56.0,1.009091,1.037037,0.0
2,2015.0,4.0,11380.0,92.0,48.0,0.968421,1.000000,2.0
3,2019.0,4.0,11200.0,105.0,57.0,0.954545,1.055556,2.0
4,2014.0,2.0,11260.0,91.0,42.0,0.989130,0.933333,3.0
...,...,...,...,...,...,...,...,...
546,2017.0,2.0,11350.0,91.0,48.0,0.938144,0.960000,3.0
547,2019.0,4.0,11500.0,94.0,50.0,0.854545,0.925926,3.0
548,2016.0,3.0,11200.0,95.0,51.0,0.989583,1.040816,2.0
549,2018.0,1.0,11620.0,99.0,50.0,0.990000,1.000000,3.0


In [ ]:
train.dtypes

year                        float64
quarter                     float64
city                        float64
Operating months            float64
Closing months              float64
Operating months_Average    float64
Closing months_Average      float64
Commercial change           float64
dtype: object

In [ ]:
train=train.astype({'year':int,
                    'quarter':int,
                    'city':int,
                    'Operating months':int,
                    'Closing months':int,
                    'Commercial change':int})
train

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average,Commercial change
0,2016,1,11290,99,51,1.031250,1.062500,0
1,2019,4,11290,111,56,1.009091,1.037037,0
2,2015,4,11380,92,48,0.968421,1.000000,2
3,2019,4,11200,105,57,0.954545,1.055556,2
4,2014,2,11260,91,42,0.989130,0.933333,3
...,...,...,...,...,...,...,...,...
546,2017,2,11350,91,48,0.938144,0.960000,3
547,2019,4,11500,94,50,0.854545,0.925926,3
548,2016,3,11200,95,51,0.989583,1.040816,2
549,2018,1,11620,99,50,0.990000,1.000000,3


In [ ]:
train.to_csv('train.csv',index=False,header=True)

In [ ]:
train_dataset=pd.read_csv('train.csv')
train_dataset

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average,Commercial change
0,2016,1,11290,99,51,1.031250,1.062500,0
1,2019,4,11290,111,56,1.009091,1.037037,0
2,2015,4,11380,92,48,0.968421,1.000000,2
3,2019,4,11200,105,57,0.954545,1.055556,2
4,2014,2,11260,91,42,0.989130,0.933333,3
...,...,...,...,...,...,...,...,...
546,2017,2,11350,91,48,0.938144,0.960000,3
547,2019,4,11500,94,50,0.854545,0.925926,3
548,2016,3,11200,95,51,0.989583,1.040816,2
549,2018,1,11620,99,50,0.990000,1.000000,3


- test.csv 만들기

In [ ]:
x_test.shape

(62, 7)

In [ ]:
test=pd.DataFrame(x_test,columns=['year','quarter','city','Operating months','Closing months', 'Operating months_Average', 'Closing months_Average'])
test

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2015.0,3.0,11320.0,89.0,45.0,0.936842,0.957447
1,2019.0,3.0,11140.0,134.0,62.0,1.229358,1.169811
2,2014.0,4.0,11170.0,103.0,49.0,1.095745,1.065217
3,2015.0,2.0,11590.0,97.0,49.0,1.021053,1.042553
4,2019.0,2.0,11710.0,98.0,49.0,0.933333,0.960784
...,...,...,...,...,...,...,...
57,2016.0,2.0,11680.0,86.0,45.0,0.905263,0.937500
58,2019.0,4.0,11530.0,110.0,52.0,1.000000,0.962963
59,2018.0,1.0,11560.0,105.0,50.0,1.050000,1.000000
60,2018.0,4.0,11500.0,89.0,48.0,0.872549,0.941176


In [ ]:
test=test.astype({'year':int,
                    'quarter':int,
                    'city':int,
                    'Operating months':int,
                    'Closing months':int})
test

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2015,3,11320,89,45,0.936842,0.957447
1,2019,3,11140,134,62,1.229358,1.169811
2,2014,4,11170,103,49,1.095745,1.065217
3,2015,2,11590,97,49,1.021053,1.042553
4,2019,2,11710,98,49,0.933333,0.960784
...,...,...,...,...,...,...,...
57,2016,2,11680,86,45,0.905263,0.937500
58,2019,4,11530,110,52,1.000000,0.962963
59,2018,1,11560,105,50,1.050000,1.000000
60,2018,4,11500,89,48,0.872549,0.941176


In [ ]:
test.to_csv('test.csv',index=False,header=True)

In [ ]:
test_dataset=pd.read_csv('test.csv')
test_dataset

,year,quarter,city,Operating months,Closing months,Operating months_Average,Closing months_Average
0,2015,3,11320,89,45,0.936842,0.957447
1,2019,3,11140,134,62,1.229358,1.169811
2,2014,4,11170,103,49,1.095745,1.065217
3,2015,2,11590,97,49,1.021053,1.042553
4,2019,2,11710,98,49,0.933333,0.960784
...,...,...,...,...,...,...,...
57,2016,2,11680,86,45,0.905263,0.937500
58,2019,4,11530,110,52,1.000000,0.962963
59,2018,1,11560,105,50,1.050000,1.000000
60,2018,4,11500,89,48,0.872549,0.941176


- solution.csv : test데이터의 라벨값

In [ ]:
y_test

array([3, 0, 0, 0, 3, 0, 0, 1, 3, 0, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 0, 0,
       0, 3, 0, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 3, 1, 0, 3, 3, 1, 1, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 2, 3, 3, 3, 3, 1, 3, 0])

In [ ]:
id=np.array([i for i in range(62)]).reshape(-1,1)
y_test=y_test.reshape(-1,1)

solution=np.hstack([id,y_test])

solution=pd.DataFrame(solution,columns=['ID','Label'])
solution

,ID,Label
0,0,3
1,1,0
2,2,0
3,3,0
4,4,3
...,...,...
57,57,3
58,58,3
59,59,1
60,60,3


In [ ]:
solution.to_csv('solution.csv',index=False,header=True)

In [ ]:
solution_data=pd.read_csv('solution.csv')
solution_data

,ID,Label
0,0,3
1,1,0
2,2,0
3,3,0
4,4,3
...,...,...
57,57,3
58,58,3
59,59,1
60,60,3


- submission.csv 만들기

In [ ]:
id=np.array([i for i in range(62)]).reshape(-1,1)
label=np.array([0 for i in range(62)]).reshape(-1,1)

submit=np.hstack([id,label])

submit=pd.DataFrame(submit,columns=['ID','Label'])
submit

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
57,57,0
58,58,0
59,59,0
60,60,0


In [ ]:
submit.to_csv('submit.csv',index=False,header=True)

In [ ]:
submit_file=pd.read_csv('submit.csv')
submit_file

,ID,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
57,57,0
58,58,0
59,59,0
60,60,0
